<a href="https://colab.research.google.com/github/merazAfridi/PneumoNet-CNN-Based-Analysis-for-Pediatric-Pneumonia-Medical-image-analysis-/blob/main/Pediatric_Pneumonia_classification_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

gazimerazmehedi_pediatric_pneumonia_path = kagglehub.dataset_download('gazimerazmehedi/pediatric-pneumonia')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Image Dimensions and Batch Size
img_height, img_width = 128, 128
batch_size = 32


In [ ]:
#Data Augmentation for Training Data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [ ]:
#Load Training Data
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/pediatric-pneumonia/train/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)


In [ ]:
#Load Validation Data
validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/pediatric-pneumonia/train/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [ ]:
# Step 6: Load Test Data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # No augmentation for test data

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/pediatric-pneumonia/test/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


In [ ]:
#Print Shapes and Sizes of Datasets
print(f'Train Data: {train_generator.n} images')
print(f'Validation Data: {validation_generator.n} images')
print(f'Test Data: {test_generator.n} images')


In [ ]:
# Display First 5 Normal and Pneumonia Training Samples
def display_samples(generator, num_samples=5):
    plt.figure(figsize=(15, 5))
    normal_count = 0
    pneumonia_count = 0

    for x_batch, y_batch in generator:
        for i in range(len(y_batch)):
            if y_batch[i] == 0 and normal_count < num_samples:
                plt.subplot(2, num_samples, normal_count + 1)
                plt.imshow(x_batch[i])
                plt.title('Normal')
                plt.axis('off')
                normal_count += 1
            elif y_batch[i] == 1 and pneumonia_count < num_samples:
                plt.subplot(2, num_samples, num_samples + pneumonia_count + 1)
                plt.imshow(x_batch[i])
                plt.title('Pneumonia')
                plt.axis('off')
                pneumonia_count += 1

            if normal_count >= num_samples and pneumonia_count >= num_samples:
                break
        if normal_count >= num_samples and pneumonia_count >= num_samples:
            break

# Display
display_samples(train_generator)


In [ ]:
# Print Total Size and Shape of Datasets
print(f'Train Data Size: {train_generator.n} images')
print(f'Validation Data Size: {validation_generator.n} images')
print(f'Test Data Size: {test_generator.n} images')

x_train, y_train = next(train_generator)
x_val, y_val = next(validation_generator)
x_test, y_test = next(test_generator)

print(f'Train Data Shape: {x_train.shape} | Labels Shape: {y_train.shape}') #each batch
print(f'Validation Data Shape: {x_val.shape} | Labels Shape: {y_val.shape}')
print(f'Test Data Shape: {x_test.shape} | Labels Shape: {y_test.shape}')


In [ ]:
#VGG16

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

#load VGG16 with pretrained weights from ImageNet, excluding the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

#Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

#custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  #Dropout layer
predictions = Dense(1, activation='sigmoid')(x)  #binary classification

#model
model = Model(inputs=base_model.input, outputs=predictions)

#summary
model.summary()


In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import Precision, AUC

# Instantiate metrics
precision_metric = Precision()
auc_metric = AUC()

# save model
checkpoint_filepath = '/kaggle/working/best_model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_auc',
    save_best_only=True,
    mode='max',
    verbose=1
)

#Compile
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', precision_metric, auc_metric])

# Fit model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20,
    callbacks=[model_checkpoint_callback]
)



In [ ]:
# Summarize history for accuracy
plt.figure(figsize=(8, 3))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



In [ ]:
# Summarize history for loss
plt.figure(figsize=(8, 3))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Summarize history for Precision
plt.figure(figsize=(8, 3))
plt.plot(history.history['precision_4'])
plt.plot(history.history['val_precision_4'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid()
plt.show()


In [ ]:
# Summarize history for AUC
plt.figure(figsize=(8, 3))
plt.plot(history.history['auc_2'])
plt.plot(history.history['val_auc_2'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# Best Training
best_train_loss = min(history.history['loss'])
best_val_loss = min(history.history['val_loss'])
best_train_accuracy = max(history.history['accuracy'])
best_val_accuracy = max(history.history['val_accuracy'])
best_train_precision = max(history.history['precision_4'])
best_val_precision = max(history.history['val_precision_4'])
best_train_auc = max(history.history['auc_2'])
best_val_auc = max(history.history['val_auc_2'])

# Print the best performing metrics
print(f'Best Training Loss: {best_train_loss:.6f}')
print(f'Best Validation Loss: {best_val_loss:.6f}')
print(f'Best Training Accuracy: {best_train_accuracy:.6f}')
print(f'Best Validation Accuracy: {best_val_accuracy:.6f}')
print(f'Best Training Precision: {best_train_precision:.6f}')
print(f'Best Validation Precision: {best_val_precision:.6f}')
print(f'Best Training AUC: {best_train_auc:.6f}')
print(f'Best Validation AUC: {best_val_auc:.6f}')


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

TEST_DIR = '/kaggle/input/pediatric-pneumonia/test/'

# Get true labels from the directory structure
true_labels = []
for filename in test_generator.filenames:
    if 'NORMAL' in filename:
        true_labels.append(0)  # Normal
    else:
        true_labels.append(1)  # Pneumonia

true_labels = np.array(true_labels)

#predictions
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int)

In [ ]:
#visualize some actual vs predicted
def visualize_predictions_grid(test_generator, predicted_classes, true_labels, num_images_per_class=10):
    #initialize images based on true labels
    normal_images = []
    pneumonia_images = []

    # Collect images based on their true labels
    for img_batch, label_batch in test_generator:
        for i in range(len(img_batch)):
            if label_batch[i] == 0 and len(normal_images) < num_images_per_class:
                normal_images.append((img_batch[i], 0))
            elif label_batch[i] == 1 and len(pneumonia_images) < num_images_per_class:
                pneumonia_images.append((img_batch[i], 1))

            # Stop collecting if reached required number of images
            if len(normal_images) >= num_images_per_class and len(pneumonia_images) >= num_images_per_class:
                break
        if len(normal_images) >= num_images_per_class and len(pneumonia_images) >= num_images_per_class:
            break

    # Combine and shuffle images
    images_to_display = normal_images + pneumonia_images
    np.random.shuffle(images_to_display)

    # Display images
    f, ax = plt.subplots(4, 5, figsize=(15, 12))  # 4 rows and 5 columns
    for i in range(len(images_to_display)):
        img, true_label = images_to_display[i]
        true_label_text = 'Normal' if true_label == 0 else 'Pneumonia'
        predicted_label_text = 'Normal' if predicted_classes[i] == 0 else 'Pneumonia'

        ax[i // 5, i % 5].imshow(img)
        ax[i // 5, i % 5].set_title(f'True: {true_label_text}\nPred: {predicted_label_text}')
        ax[i // 5, i % 5].axis('off')

    plt.tight_layout()
    plt.show()
#visualize
visualize_predictions_grid(test_generator, predicted_classes, true_labels)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix

#true labels from the test generator
true_classes = test_generator.classes

#confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(4, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='magma', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
#Calculate metrics
accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes)
auc = roc_auc_score(true_labels, predictions)  # Use predicted probabilities for AUC

#Print the metrics
print(f"TEST Accuracy: {accuracy:.4f}")
print(f"TEST Precision: {precision:.4f}")
print(f"TEST AUC: {auc:.4f}")


In [ ]:
#plot metrics
def plot_metrics(accuracy, precision, auc):
    metrics = ['Accuracy', 'Precision', 'AUC']
    values = [accuracy, precision, auc]

    plt.figure(figsize=(4, 5))
    bars = plt.bar(metrics, values, color=['slateblue', 'darkcyan', 'crimson'])

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 4), ha='center', va='bottom')

    plt.ylim(0, 1)
    plt.title('Test Data Metrics')
    plt.ylabel('Score')
    plt.grid(axis='y')
    plt.show()

# visualize metrics
plot_metrics(accuracy, precision, auc)